# Implementación de API REST

# ¿Qué es REST?

> La **Transferencia de Estado Representacional** (Representational State Transfer) o **REST** es un estilo de arquitectura software para sistemas hipermedia distribuidos como la World Wide Web.

Existen, otros tipos de métodos para implementar servicios web, como **RPC**, **SOAP** o **WSDL**. Sin embargo, el uso de dichos mecanismos no se suele recomendar en favor de RESTful, ya que RESTful es mucho más fácil de entender e implementar.

Características del diseño de la arquitectura REST:
1. **Cliente/Servidor** separación clara entre los dos agentes básicos en el intercambio de información.

1. **Stateless** el servidor no tiene porqué almacenar datos del cliente para mantener un estado del mismo.

1. **Cacheable** el servidor debe poder definir algún modo de cachear dichas peticiones, para aumentar el rendimiento, escalabilidad, etc.

1. **Sistema por capas** el cliente no necesita saber cuales son las capas implicadas en la obtención de la información.

1. **Interfaz uniforme** la interfaz de comunicación entre un cliente y el servidor no debe depender ni del servidor ni del cliente, se debe cumplir una interfaz definida de antemano

Estos principios de diseño ya vienen dados por el uso del protocolo **HTTP** ya que:

* **Identificación de recursos** una aplicación REST debe poder identificar sus recursos de manera uniforme. HTTP implementa esto usando las llamadas URIs o las tradicionales [URLs](https://es.wikipedia.org/wiki/Localizador_de_recursos_uniforme).

* **Recursos y representaciones:** REST define también la manera en que podemos interactuar con la representación de un recurso, ya sea para crearlo, obtenerlo, editarlo o borrarlo. HTTP define distintos verbos y un contenido en la respuesta lo que permite modelar las operaciones con el recurso en el caso de que tengamos permiso para hacerlo.

* **Mensajes autodescriptivos** cuando se hacen peticiones a un servidor, éste debería devolver una respuesta que permita entender sin lugar a duda cual ha sido el resultado de la operación, así como si dicha operación es cacheable, si ha habido algún error, etc. HTTP implementa esto a través del estado y una serie de cabeceras. El uso de las cabeceras depende de la implementación, REST no fuerza el contenido de las cabeceras, aunque existen algunas convenciones [básicas](http://www.restapitutorial.com/httpstatuscodes.html).

* **HATEOAS** es la necesidad de incluir en las respuestas del servidor toda aquella información que necesita el cliente para seguir operando con el servicio web. La mayoría de servicios web no cumplen con esta necesidad. 

```json
{
    "id": 4,
    "nombre": "Juan",
    "apellido": "Carlos",
    "coches": [
    	{
    		"id": 1253
    	}
    ]
}
```

```json
{
    "id": 4,
    "nombre": "Juan",
    "apellido": "Carlos",
    "coches": [
    	{
    		"coche": "http://miservidor/concesionario/api/v1/clientes/4/coches/1253"
    	}
    ]
}
```

# ¿Qué es un servicio web RESTful?

Un servicio web RESTful hace referencia a un servicio web que implementa la arquitectura REST.

Un servicio web RESTful contiene lo siguiente:

* **URI del recurso** por ejemplo http://servidor.com/api/nombres/1.
* **El tipo de la representación de dicho recurso** devolver la cabecera *Content-type* con alguno de los formatos más comunes JSON, XML y TXT.
* **Operaciones soportadas** definir correctamente el uso de los verbos 
GET, PUT, POST, DELETE, PATCH.
* **Hipervínculos** es deseable incluir los enlaces o vínculos hacia otras acciones como parte de la respuesta.

Luego de ver como se relaciona REST con el protocolo HTTP un ejemplo de diseño de **endpoints** adecuados puede ser:

* **GET** - Obtener información sobre un recurso 
  * http://api.com/api/orders (recuperar la lista de pedidos)

* **GET** - Obtener información sobre un recurso
  * http://api.com/api/orders/123 (recuperar orden # 123)
  
* **POST** - Crear un nuevo recurso
  * http://api.com/api/orders (crear un nuevo pedido, a partir de los datos proporcionados con la solicitud)
  
* **PUT** - Actualizar un recurso 
  * http://api.com/api/orders/123 (actualización de la orden # 123, a partir de los datos proporcionados con la solicitud)
  
* **DELETE** - Eliminar un recurso
  * http://api.com/api/orders/123 (borrar pedido # 123)
  
El diseño REST no específica como se proporcionan los datos al servidor y generalmente se serializan en el cuerpo de la solicitud como JSON, o a veces como argumentos en el fragmento **query** del URL.

# Servicio web sencillo en Flask

(Api Flask)[https://blog.miguelgrinberg.com/post/designing-a-restful-api-with-python-and-flask]

```python
#!flask/bin/python
from flask import Flask, jsonify, abort, make_response

app = Flask(__name__)

tasks = [
    {
        'id': 1,
        'title': 'Aprender Python',
        'description': 'Buscar material en internet para aprender el lenguaje Python', 
        'done': False
    },
    {
        'id': 2,
        'title': 'Hacer una API',
        'description': 'Crear un servicio RESTful en base a un data set de http://datos.gob.ar/', 
        'done': False
    }
]

# Manejador de errores
@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)

# Get de coleccion y de recurso
@app.route('/todo/api/v1.0/tasks', methods=['GET'])
def get_tasks():
    return jsonify({'tasks': tasks})

@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    return jsonify({'task': task[0]})

# Crear un recurso
@app.route('/todo/api/v1.0/tasks', methods=['POST'])
def create_task():
    if not request.json or not 'title' in request.json:
        abort(400)
    task = {
        'id': tasks[-1]['id'] + 1,
        'title': request.json['title'],
        'description': request.json.get('description', ""),
        'done': False
    }
    tasks.append(task)
    return jsonify({'task': task}), 201

# Actualizar un recurso
@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    if not request.json:
        abort(400)
    if 'title' in request.json and type(request.json['title']) != unicode:
        abort(400)
    if 'description' in request.json and type(request.json['description']) is not unicode:
        abort(400)
    if 'done' in request.json and type(request.json['done']) is not bool:
        abort(400)
    task[0]['title'] = request.json.get('title', task[0]['title'])
    task[0]['description'] = request.json.get('description', task[0]['description'])
    task[0]['done'] = request.json.get('done', task[0]['done'])
    return jsonify({'task': task[0]})

# Eliminar un recurso
@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    tasks.remove(task[0])
    return jsonify({'result': True})

if __name__ == '__main__':
    app.run(debug=True)
```

# Técnicas avanzadas

* **Versioning** Por la propia naturaleza de una API, cuanto ha salido a producción al menos 1 usuario va a hacer uso de ella de manera continuada, a pesar de que en el futuro se generen nuevas versiones de la API que sustituyen a la versión actual. En estos casos, se debe disponer de algún mecanismo que permita a los usuarios acceder a una versión u otra. La solucion mas obvia es cambiar la URL base de los recursos.

* **Autenticación** Esto es vital para mantener un mínimo de seguridad sobre las operaciones. Cuando se trata de autentificación, un método que está de moda últimamente, y que se ha probado robusto y estable, es la autentificación OAuth, en especial la versión 2.

* **Paginación de resultados** Normalmente cuando se muestran datos a los usuarios en sitios web, se utiliza algún tipo de paginación, ya que no resulta muy práctico mostrar miles de registros de una vez, aparte de ser ineficiente en cuanto a recursos del servidor. El mismo principio se aplica también en el diseño de APIs, en dónde el resultado de una petición sobre un recurso que cuenta con miles de entradas debe estar paginado.

# Para ver y trabajar en clase

Terminamos la implementación iniciada en clase y fijamos las bases para el trabajo final. 

* [Flask-RESTful](https://flask-restful.readthedocs.io/en/latest/)
* [flask-restful-swagger](https://github.com/rantav/flask-restful-swagger)
* [A RESTful Tutorial](http://www.restapitutorial.com/)

# Ejercicio Final

Aplicar lo visto a otro [dataset](http://datos.gob.ar)
  
* Objtener datos
* Filtrar, Mappear y Reducir
* Publicar una API (GET, POST, PUT y DELETE)
* Inverstigar otro mecanismo de persistencia y opcionalmente un ORM